# 1. Tensorflow基本概念

* 使用图（graphs）来表示计算任务
* 在被称之为会话（Session）的上下文（context）中执行图
* 使用tensor表示数据
* 通过变量（Variable）维护状态
* 使用feed和fetch可以为任意的操作赋值或者从其中获取数据

Tensorflow是一个编程系统，使用图（graphs）来表示计算任务，图（graphs）中的节点称之为op （operation），一个op获得0个或多个Tensor，执行计算，产生0个或多个Tensor。Tensor 看作是 一个 n 维的数组或列表。图必须在会话（Session）里被启动。

# 2. Tensorflow结构
![](http://upload-images.jianshu.io/upload_images/1791718-9ea30c056280bdb4.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 2.1 创建图，启动图 

In [1]:
import tensorflow as tf

In [5]:
# 创建一个常量op，一行两列的矩阵[[3, 3]]
m1 = tf.constant([[3, 3]])

# 创建一个常量op，两行一列的矩阵
m2 = tf.constant([[2], [3]])

# 创建一个矩阵乘法的op
product = tf.matmul(m1, m2)

# 只是定义了op，但并没有放在session中执行，所以不会输出15
print(product)

<bound method Tensor.get_shape of <tf.Tensor 'MatMul_2:0' shape=(1, 1) dtype=int32>>


In [9]:
# 定义一个session，启动默认的图
sess = tf.Session()

# 调用sess的run方法来执行乘法op，run(product)触发图中3个op
result = sess.run(product)
print(result)

# 关闭会话
sess.close()

[[15]]


In [ ]:
# 用with就不用手动关闭会话
with tf.Session() as sess:
    result = sess.run(product)
    print(result)

## 2.2 TensorFlow变量


In [ ]:
import tensorflow as tf

In [12]:
x = tf.Variable([1, 2])
a = tf.constant([3, 3])

# shift+tab 快捷键可以查看函数定义
sub = tf.subtract(x, a)
add = tf.add(x, sub)

# 注意要对变量进行初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(sub))
    print(sess.run(add))

[-2 -1]
[-1  1]


In [18]:
# 创建一个变量初始化为0，给变量取名counter
state = tf.Variable(0, name='counter')

# 创建一个op，使state加1
new_value = tf.add(state, 1)

# 调用一个赋值的操作，而不能直接使用等号赋值
update = tf.assign(state, new_value)

# 使用到了Variable变量，需要初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(state)) # 初始化为0，打印0
    for _ in range(5):
        sess.run(update)
    print(sess.run(state))

0
5


## 2.3 Fetch & Feed

In [20]:
# Fetch: 同时运行多个op
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)

add = tf.add(input2, input3)
mul = tf.multiply(input1, add)

with tf.Session() as sess:
    # 在sess中同时运行mul和add两个op
    result = sess.run([mul, add])
    print(result)
    

[21.0, 7.0]


In [21]:
# Feed
# 创建占位符
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    # feed的数据以字典的形式传入
    print(sess.run(output, feed_dict={input1: 7.0, input2: 2.0}))


14.0


## 2.4 TensorFlow简单示例

In [22]:
import tensorflow as tf
import numpy as np

In [23]:
# 使用numpy生成100个随机点
x_data = np.random.rand(100)
# 直线：斜率0.1，截距0.2
y_data = x_data * 0.1 + 0.2

# 构造一个线性模型
b = tf.Variable(0.)
k = tf.Variable(0.)
# b和k是变量，初始化为0
y = k * x_data + b

# 二次代价函数
# reduce_mean求均值，真实值y_data减去预测值y的平方再求平均值
loss = tf.reduce_mean(tf.square(y_data - y))

# 定义一个梯度下降法来训练一个优化器
# 学习率0.2 shift+tab查看
optimizer = tf.train.GradientDescentOptimizer(0.2)

# 最小化代价函数（训练的目的）
train = optimizer.minimize(loss)

# 用了Variable就要init
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    # 201次迭代
    for step in range(201):
        sess.run(train)
        # 每20次迭代打印一下
        if step % 20 == 0:
            print(step, sess.run([k, b]))

0 [0.050500482, 0.099160887]
20 [0.10092723, 0.19952722]
40 [0.10056911, 0.19970989]
60 [0.10034928, 0.19982195]
80 [0.10021436, 0.19989073]
100 [0.10013156, 0.19993295]
120 [0.10008074, 0.19995885]
140 [0.10004956, 0.19997475]
160 [0.10003041, 0.19998451]
180 [0.10001865, 0.1999905]
200 [0.10001145, 0.19999416]
